In [17]:
import omf
import pandas as pd
import numpy as np

In [18]:
csv_file = "E:\\MDA\\Beta Testing\\MIOPP\\Block Model - Mining Model\\test_bm.csv"
df = pd.read_csv(csv_file)
df = df.sort_values(by=["I", "J", "K"], ascending=[True, True, True])
df

,X,Y,Z,classification_no,I,J,K,size (X),size(Y),size(Z),...,co,cu,density,domain,exclusion_zone,geotech_zone,ni,percentage,strength,weather
0,295270.0,6796670.0,-395.0,1,1,1,1,20.0,20.0,10.0,...,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
512,295270.0,6796670.0,-385.0,1,1,1,2,20.0,20.0,10.0,...,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
64,295270.0,6796670.0,-375.0,1,1,1,3,20.0,20.0,10.0,...,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
576,295270.0,6796670.0,-365.0,1,1,1,4,20.0,20.0,10.0,...,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
8,295270.0,6796670.0,-355.0,1,1,1,5,20.0,20.0,10.0,...,0.001,0.001,2.5,ng,0,40,0.001,1.0,Strong,Fresh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469659,297550.0,6799490.0,455.0,1,115,142,86,20.0,20.0,10.0,...,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
1469662,297550.0,6799490.0,465.0,1,115,142,87,20.0,20.0,10.0,...,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
1469663,297550.0,6799490.0,475.0,1,115,142,88,20.0,20.0,10.0,...,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air
1469698,297550.0,6799490.0,485.0,1,115,142,89,20.0,20.0,10.0,...,0.000,0.000,0.0,ng,0,40,0.000,0.0,AIR,Air


In [19]:
bm_origin = [6796660, 295260, -400]
min_y, max_y = 6796660, 6799500
min_x, max_x = 295260, 297560
min_z, max_z = -400, 500
block_size = 20.
block_size_z = 10.
differenceX = max_x - min_x
differenceY = max_y - min_y
differenceZ = max_z - min_z
num_blocksX = differenceX // block_size
num_blocksY = differenceY // block_size
num_blocksZ = differenceZ // block_size_z

tensor_u = np.full(int(num_blocksX), block_size, dtype=float)
tensor_v = np.full(int(num_blocksY), block_size, dtype=float)
tensor_w = np.full(int(num_blocksZ), block_size_z, dtype=float)


In [20]:
bm_attributes = []

for element in ["cu", "ni", "density", "co"]:
  if element in df.columns:
      element_array = df[element].astype(float).to_numpy()
      attr = omf.ScalarData(
         array = np.array(element_array), name =element,  location='cells', )
      bm_attributes.append(attr)

In [21]:
proj = omf.Project(
    name='Menkar',
    description='Test OMF BM'
)
vol = omf.VolumeElement(
    name='Test BM',
    geometry=omf.VolumeGridGeometry(
        tensor_u=tensor_u,
        tensor_v=tensor_v,
        tensor_w=tensor_w,
        origin=bm_origin
    ),
    data=bm_attributes
)

In [22]:
all_vertex_coords = [[0,0,0], [0,1,0], [1,1,0], [1,0,-0.1]]
triangles = [[0,1,2],[2,3,0]]

In [23]:
grid = omf.SurfaceElement(
    name='trisurf',
    geometry=omf.SurfaceGeometry(
        vertices=np.array(all_vertex_coords),
        triangles=np.array(triangles)
    ),
)

In [24]:
proj.elements = [vol, grid]
assert proj.validate()
omf.OMFWriter(proj, '../design_assets/bm_from_csv.omf')